# Introduction
This notebook demonstrates the integration capabilities of roboquant with OANDA. OANDA is a broker with a focus on FOREX/CFDs. 

There are currently four types of integration with OANDA available:
1. Retrieve historic market data from OANDA, can be used during back testing 
2. Subscribe to live market data from OANDA, cna be used during live testing and paper trading
3. Paper trading broker on the OANDA platform (WIP)
4. Exchange rates from OANDA

In [ ]:
%use @http://roboquant.org/roboquant-extra.json

// By default the required OANDA classes are not imported, so we need to do that first. 
// This import will load the various Feed and Broker classes for integrating with OANDA.
import org.roboquant.oanda.*

// By default USD, EUR and GBP are displayed with 2 decimals, but for FOREX trading we want more precision 
// in order to see small differences. So we increase the digits for all registered currencies by 3
Currency.increaseDigits(3)

Welcome()

# Back Testing
The first example shows how to use historic data from OANDA in a back test. To use Oanda you'll need to authenticate with a key and optional specify the account Id. There are several ways to pass them to the feed, either directly or through system environment or a `.env` property file. 


<div class="alert alert-block alert-info">
Make sure you use always the demo account since roboquant trading functionality is still very much in Beta
</div>

In [ ]:
 // Get the oanda.key from a .env file
val feed = OANDAHistoricFeed()
println(feed.availableAssets.summary("available assets"))

// Specify the key directly. Be careful not to expose this key to third parties
// val feed = OANDAHistoricFeed() { key = "my_secret_key" }

// Retrieve PriceBars for 1 day
val timeframe = Timeframe.parse("2021-08-10", "2021-08-11")
val symbols = listOf("EUR_USD", "GBP_USD", "AUD_USD").toTypedArray()
feed.retrieve(*symbols, timeframe = timeframe)

feed.assets.summary()

In [ ]:
PriceBarChart(feed, feed.assets.first())

We'll define a strategy of which the buy and sell signals are based on candlestick patterns. Although trading Forex is like any another asset class, there are some configuration parameters that deviate from assets classes (less spread, more shorting)
OANDA.roboquant presets these parameters

In [ ]:
val strategy = TaLibStrategy(15)

// We want to generate a BUY signal if we detect a Three White Soldiers pattern
strategy.buy { cdl3WhiteSoldiers(it) }

// We want to generate a SELL signal if we detect the candlestick pattern Three Black Crows 
strategy.sell { cdl3BlackCrows(it) }

val roboquant = OANDA.roboquant(strategy, AccountMetric())

In [ ]:
roboquant.run(feed)

## Results

In [ ]:
val account = roboquant.broker.account
account.fullSummary()

In [ ]:
val metric = roboquant.logger.getMetric("account.equity")
MetricChart(metric)

In [ ]:
feed.assets.forEach { 
    PriceBarChart(feed, it, account.trades).render()
}

# Live Testing
Now we create an instance of the OANDALiveFeed and subscribe to same three forex pairs as before. If you run these cells when FOREX exchanges are closed, there will no orders and trades of course.

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook(*symbols)

In [ ]:
val strategy = EMAStrategy()
val eventRecorder = EventCaptureMetric()
val roboquant = OANDA.roboquant(strategy, eventRecorder, AccountMetric())

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. We'll run it for 5 minutes, but you can change this. 

In [ ]:
val timeframe = Timeframe.next(60.minutes)
roboquant.run(feed, timeframe)

## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
val account = roboquant.broker.account
account.fullSummary()

In [ ]:
for (asset in feed.assets) PriceChart(eventRecorder, asset, account.trades).render()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
val logger = roboquant.logger
logger.metricNames.summary()

In [ ]:
val accountValue = logger.getMetric("account.equity")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeAssetChart(account.trades)

# Paper Trading
And just as it was the case in the previous section, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated. Besides the paper trading, this example also uses the Exchange Rates from OANDA to support multi-currency trading.

<div class="alert alert-block alert-warning">
The integration with the OANDA trading and broker API is still very much work in progress, so only use this with a paper trading account
</div>

In [ ]:
Config.exchangeRates = OANDAExchangeRates()
val broker = OANDABroker()
val account = broker.account

In [ ]:
// This provides an overview of your demo account portfolio at OANDA
// before we started the actual paper trading run
account.summary()

## Run
Get the data we want to use to feed our strategy. The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. This time we'll run it for 5 minutes.

In [ ]:
val feed = OANDALiveFeed()
feed.subscribeOrderBook(*symbols)

In [ ]:
val strategy = EMAStrategy.PERIODS_5_15 // Use EMA Crossover strategy
val policy = FlexPolicy(shorting = true) // We want to allow for short positions if we do FOREX trading

val eventRecorder = EventCaptureMetric()
val roboquant = Roboquant(strategy, AccountMetric(), eventRecorder, policy = policy, broker = broker)

In [ ]:
val timeframe = Timeframe.next(60.minutes) // restrict the time we will run the paper trading
roboquant.run(feed, timeframe)

## Results

In [ ]:
val account = roboquant.broker.account
account.fullSummary()

In [ ]:
val data = roboquant.logger.getMetric("account.equity")
MetricChart(data)

In [ ]:
for (asset in feed.assets) PriceChart(eventRecorder, asset, account.trades).render()

In [ ]:
TradeChart(account.trades)